In [61]:
import glob, os, json
from pathlib import Path
import tqdm
import spacy, pyinflect, nltk
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from itertools import chain
from collections import Counter

In [4]:
# nltk.download("wordnet")
# nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alexstratton2121/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/alexstratton2121/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
def split(line):
    separated = line.split(",", 1)
    return (separated[0], separated[1])

In [3]:
def similar_words(word):
    nlp = spacy.load("en_core_web_sm")
    similar, lemmas = [], []
    for word_set in wn.synsets("need", pos=wn.VERB):
        for related in word_set.lemma_names():
            similar.append(related)
        
        for lemma in word_set.lemmas():
            lemmas.append(lemma)
        
    verbs = ' '.join(set(similar))
    spacy_doc = nlp(verbs)
    verb_types = ["VB", "VBG", "VBD", "VBN", "VBP", "VBZ"]
    verbs = [(token._.inflect(verb_type), verb_type) for token in spacy_doc 
                  for verb_type in verb_types if '_' not in str(token)]
    
    nouns = []
    for lemma in lemmas:
        for related_form in lemma.derivationally_related_forms():
            nouns.append(related_form.name().split('.', 1)[0])
    nouns = ' '.join(set(nouns))
    spacy_doc = nlp(nouns)
    noun_types = ["NN", "NNS", "NNP", "NNPS"]
    nouns = [(token._.inflect(noun_type), noun_type) for token in spacy_doc 
                  for noun_type in noun_types]
    nouns = [noun for noun in nouns if noun[0]]
    
    return verbs + nouns

In [31]:
def frequency_detection_0(sentence, needs_list, most_similar):
    pos_tagged = pos_tag(sentence.split())
    if len([word for word in pos_tagged if word in most_similar]) != 0:
        counter = Counter([need[0] for need in pos_tagged
                           if (need[0] in needs_list and need[1] in ["NN", "NNS", "NNP", "NNPS"])])
        return counter
    
    return Counter()

In [32]:
def frequency_detection_1(sentence, needs_list):
    pos_tagged = pos_tag(sentence.split())
    counter = Counter([need[0] for need in pos_tagged
                           if (need[0] in needs_list and need[1] in ["NN", "NNS", "NNP", "NNPS"])])
    return counter

In [5]:
# Priority needs list for expanded data set
priority_needs = []
with open("priority_needs_all.txt", 'r') as f:
    for line in f:
        priority_needs.append(line.strip())
        
priority_needs[33] = priority_needs[33][:3] + 'i' + priority_needs[33][3:]

In [62]:
all_files = sorted(glob.glob(os.path.join(str(Path.cwd()) + '/gsData', "*.txt")))
sim_words = similar_words("need")
for file in all_files:
    needs_for_day = []
    with open(file) as f:
        tweets_for_day = [line.replace('\n', '') for line in f]
        needs_for_day = [frequency_detection_0(sent, priority_needs, sim_words) for sent in tweets_for_day]
        needs_for_day = sum([x for x in needs_for_day if x], Counter())
        with open(str(Path.cwd()) + "/needs_frequency_information/" + file[51:-4] + "_frequency.json", 'w') as o:
            json.dump(dict(needs_for_day), o)

KeyboardInterrupt: 